In [1]:
import numpy as np
import os
import pandas as pd
from arcpy import env
from arcpy import TableToTable_conversion
import matplotlib.ticker as mtick
from IPython.display import display, Markdown

#import arcgis libraries
from arcgis.gis import *
gis = GIS()

In [2]:
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))

In [59]:
dirWork      = os.getcwd()
dirData      = os.path.join(dirWork, r'data')
dirTDM       = os.path.join(dirData, r'tdm\1_WF')
dirResults   = os.path.join(dirWork, r'results')
dirLayers    = os.path.join(dirResults, r'layers')
dirProcessed = os.path.join(dirWork, r'data\tdm\1_WF')

In [4]:
dDirections = {
     'Dir'     : ['D1'   ,'D2'   ],
     'DirDesc' : ['NB/EB','SB/WB']
     
}
df_Directions = pd.DataFrame(dDirections)


In [5]:
shp_Forecasts = os.path.join(dirData,  r'forecasts\WFRCForecasts_2019RTP_v831_20200508_StateWideMerged_20200608h.shp')
                        
#import into spatially-enabled DataFrame
sdf_Forecasts = pd.DataFrame.spatial.from_featureclass(shp_Forecasts)

In [100]:
df_Scenarios       = pd.read_csv(os.path.join(dirResults, r'scenarios.csv'))
df_VCGroup_Seasons = pd.read_csv(os.path.join(dirResults, r'vcgroups_seasons.csv'))
df_VCGroup_DOWPeak = pd.read_csv(os.path.join(dirResults, r'vcgroups_dowpeak.csv'))
df_VCGroups        = pd.read_csv(os.path.join(dirResults, r'vcgroups.csv'))

df_VCGroups_15  = df_VCGroups[['VCGroupCode']]
df_VCGroups_Hr  = df_VCGroups[['VCGroupCode']]
df_VCGroups_Prd = df_VCGroups[df_VCGroups['HrPctOf'] == 'Prd'].copy()
df_VCGroups_Prd = df_VCGroups_Prd[['VCGroupCode']]
df_VCGroups_15 ['VCGroupCode'] = df_VCGroups_15['VCGroupCode'] + "15"
df_VCGroups_Hr ['VCGroupCode'] = df_VCGroups_Hr['VCGroupCode'] + "Hr"
df_VCGroups_Prd['VCGroupCode'] = df_VCGroups_Prd['VCGroupCode'] + "Prd"
df_VCGroups_Expanded = pd.concat([df_VCGroups_15,df_VCGroups_Hr,df_VCGroups_Prd])
    
display(df_Scenarios)
display(df_VCGroup_Seasons)
display(df_VCGroup_DOWPeak)
display(df_VCGroups_Expanded)


C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,Process,ScenarioCode,ScenarioGroup,ScenarioName,ScenarioYear,SegSummaryCSV,PrvScenarioCode,AdjScenarioCode
0,No,Base15,Base,2015 Base,2015,v831_SE15_Net15_Summary_SEGID_Detailed.csv,NaN,NaN
1,No,Base19,Base,2019 Base,2019,v831_SE19_Net19_Summary_SEGID_Detailed.csv,Base15,NaN
2,No,TIP24,TIP,2024 TIP,2024,v831_SE24_Net24_Summary_SEGID_Detailed.csv,Base19,NaN
3,No,RTP30,RTP,2030 RTP,2030,v831_SE30_Net30_Summary_SEGID_Detailed.csv,TIP24,NaN
4,No,RTP40,RTP,2040 RTP,2040,v831_SE40_Net40_Summary_SEGID_Detailed.csv,RTP30,NaN
5,No,RTP50,RTP,2050 RTP,2050,v831_SE50_Net50_Summary_SEGID_Detailed.csv,RTP40,NaN
6,No,NB30,No Build,2030 No Build,2030,v831_SE30_Net24_Summary_SEGID_Detailed.csv,TIP24,RTP30
7,No,NB40,No Build,2040 No Build,2040,v831_SE40_Net24_Summary_SEGID_Detailed.csv,NB30,RTP40
8,No,NB50,No Build,2050 No Build,2050,v831_SE50_Net24_Summary_SEGID_Detailed.csv,NB40,RTP50
9,No,UF30,Needs,2030 Needs,2030,v831_SE30_Net30UF_Summary_SEGID_Detailed.csv,TIP24,RTP30


,SeasonCode,SeasonGroup,SeasonName,SeasonType
0,Ann,S00-Ann,Annual,Season
1,Win,S01-Win,Winter,Season
2,Spr,S02-Spr,Spring,Season
3,Sum,S03-Sum,Summer,Season
4,Fal,S04-Fal,Fall,Season
5,Jan,M01-Jan,January,Month
6,Feb,M02-Feb,February,Month
7,Mar,M03-Mar,March,Month
8,Apr,M04-Apr,April,Month
9,May,M05-May,May,Month


,DOWGroup,DOWPkCode,DOWPkName,HrPctOf,Prd
0,W1-Weekday,WkAM,AM - Weekday,Prd,AM
1,W1-Weekday,WkMD,Midday - Weekday,Prd,MD
2,W1-Weekday,WkPM,PM - Weekday,Prd,PM
3,5-Friday,FrPM,PM - Friday,DY,PM
4,6-Saturday,SaDY,Saturday,DY,DY
5,7-Sunday,SuDY,Sunday,DY,DY


,VCGroupCode
0,AnnWkAM15
1,WinWkAM15
2,SprWkAM15
3,SumWkAM15
4,FalWkAM15
...,...
46,AugWkPMPrd
47,SepWkPMPrd
48,OctWkPMPrd
49,NovWkPMPrd


In [7]:
df_VCGroups_Prd

,VCGroupCode
0,AnnWkAMPrd
1,WinWkAMPrd
2,SprWkAMPrd
3,SumWkAMPrd
4,FalWkAMPrd
5,JanWkAMPrd
6,FebWkAMPrd
7,MarWkAMPrd
8,AprWkAMPrd
9,MayWkAMPrd


In [8]:
#define functions to get scenario data
def idFromCode (sCode):
    return df_Scenarios.index[df_Scenarios['ScenarioCode'] == sCode].tolist()[0]
def yrFromCode (sCode):
    return df_Scenarios[df_Scenarios['ScenarioCode'] == sCode].iloc[0]['ScenarioYear']
def codeFromID (sID):
    return df_Scenarios.loc[sID]['ScenarioCode']
def yearFromID (sID):
    return df_Scenarios.loc[sID]['ScenarioYear']
def ScenarioNameFromCode (sCode):
    return df_Scenarios[df_Scenarios['ScenarioCode'] == sCode].iloc[0]['ScenarioName']


def DOWPkNameFromCode (sCode):
    return df_VCGroup_DOWPeak[df_VCGroup_DOWPeak['DOWPkCode'] == sCode].iloc[0]['DOWPkName']
def SeasonNameFromCode (sCode):
    return df_VCGroup_Seasons[df_VCGroup_Seasons['SeasonCode'] == sCode].iloc[0]['SeasonName']



#quick tests of functions
#display(idFromCode('50RTP'))
#display(yrFromCode('50RTP'))
#display(codeFromID(4))
#display(yearFromID(4))
#display(df_Scenarios.loc[7]['ScenarioCode'])

In [9]:
sCode = 'AvgWkPM15'
sCode = 'WkPM'
#DOWPkNameFromCode ()
#sCode
df_VCGroup_DOWPeak

,DOWGroup,DOWPkCode,DOWPkName,HrPctOf,Prd
0,W1-Weekday,WkAM,AM - Weekday,Prd,AM
1,W1-Weekday,WkMD,Midday - Weekday,Prd,MD
2,W1-Weekday,WkPM,PM - Weekday,Prd,PM
3,5-Friday,FrPM,PM - Friday,DY,PM
4,6-Saturday,SaDY,Saturday,DY,DY
5,7-Sunday,SuDY,Sunday,DY,DY


In [10]:
dfs_VC_MaxDirFG_wVCGroups =[]

display ('Scenario Codes:')

for (idx, row) in df_Scenarios.iterrows():    

    print(row.loc['ScenarioCode'])
    
    dfs_VC_MaxDirFG_wVCGroups.append(pd.read_csv(os.path.join(dirResults, r'temp\3_VC_MaxDirFG_wVCGroups_' + row.loc['ScenarioCode'] + '.csv')))


'Scenario Codes:'

Base15
Base19
TIP24
RTP30
RTP40
RTP50
NB30
NB40
NB50
UF30
UF40
UF50
S550


In [11]:
dfs_VC_MaxDirFG_wVCGroups[0]

,SEGID,FT,AnnFrPM15,AnnSaDY15,AnnSuDY15,AnnWkAM15,AnnWkMD15,AnnWkPM15,AprFrPM15,AprSaDY15,...,SepWkPMPrd,SprWkAMPrd,SprWkMDPrd,SprWkPMPrd,SumWkAMPrd,SumWkMDPrd,SumWkPMPrd,WinWkAMPrd,WinWkMDPrd,WinWkPMPrd
0,0006_146.9,2.0,0.09,0.09,0.09,0.05,0.05,0.07,0.11,0.09,...,0.06,0.04,0.04,0.06,0.04,0.04,0.06,0.03,0.03,0.05
1,0006_149.9,2.0,0.18,0.17,0.16,0.10,0.09,0.13,0.17,0.16,...,0.11,0.07,0.07,0.10,0.09,0.08,0.12,0.06,0.06,0.09
2,0006_150.6,2.0,0.18,0.17,0.16,0.10,0.09,0.13,0.17,0.16,...,0.11,0.07,0.07,0.11,0.09,0.08,0.13,0.06,0.06,0.09
3,0006_152.6,3.0,0.17,0.16,0.16,0.12,0.09,0.14,0.19,0.17,...,0.12,0.09,0.07,0.12,0.10,0.07,0.12,0.08,0.05,0.10
4,0006_152.9,2.0,0.21,0.20,0.18,0.17,0.10,0.18,0.20,0.19,...,0.16,0.12,0.08,0.15,0.14,0.09,0.17,0.10,0.07,0.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3577,WFRC_8208,5.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3578,WFRC_8209,4.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3579,WFRC_8213,5.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3580,WFRC_8215,4.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [12]:
sScenario = 'RTP50'
sVCGroup  = 'AnnWkPMPrd'

#expression to classify enrollment
arcade_expression = ("var v = $feature." + sVCGroup + ";"
                     "var ft = $feature.FT;"
                     "if      (v<0.7  && ft<10) { return 'class_a1'; }"
                     "else if (v<0.85 && ft<10) { return 'class_a2'; }"
                     "else if (v<1.0  && ft<10) { return 'class_a3'; }"
                     "else if (v<1.5  && ft<10) { return 'class_a4'; }"
                     "else if (v>=1.5 && ft<10) { return 'class_a5'; }"
                     "else if (v<0.7  && ft>10) { return 'class_f1'; }"
                     "else if (v<0.85 && ft>10) { return 'class_f2'; }"
                     "else if (v<1.0  && ft>10) { return 'class_f3'; }"
                     "else if (v<1.5  && ft>10) { return 'class_f4'; }"
                     "else if (v>=1.5 && ft>10) { return 'class_f5'; }"
                    )

#symbology for enrollment classes
uv = [{"value":"class_a1", "label":"Arterial Less than 0.70", "symbol":{"type":"esriSLS","color":[  0,255,  0,160], "width":1.5,"style":"esriSLSSolid"}},
      {"value":"class_a2", "label":"Arterial 0.70 to 0.85"  , "symbol":{"type":"esriSLS","color":[169,243,106], "width":1.5,"style":"esriSLSSolid"}},
      {"value":"class_a3", "label":"Arterial 0.85 to 1.00"  , "symbol":{"type":"esriSLS","color":[255,228,105], "width":1.5,"style":"esriSLSSolid"}},
      {"value":"class_a4", "label":"Arterial 1.00 to 1.50"  , "symbol":{"type":"esriSLS","color":[255,  0,  0], "width":1.5,"style":"esriSLSSolid"}},
      {"value":"class_a5", "label":"Arterial More than 1.50", "symbol":{"type":"esriSLS","color":[115,  0,  0], "width":1.5,"style":"esriSLSSolid"}},
      {"value":"class_f1", "label":"Freeway Less than 0.70" , "symbol":{"type":"esriSLS","color":[  0,255,  0,160], "width":4.0,"style":"esriSLSSolid"}},
      {"value":"class_f2", "label":"Freeway 0.70 to 0.85"   , "symbol":{"type":"esriSLS","color":[169,243,106], "width":4.0,"style":"esriSLSSolid"}},
      {"value":"class_f3", "label":"Freeway 0.85 to 1.00"   , "symbol":{"type":"esriSLS","color":[255,228,105], "width":4.0,"style":"esriSLSSolid"}},
      {"value":"class_f4", "label":"Freeway 1.00 to 1.50"   , "symbol":{"type":"esriSLS","color":[255,  0,  0], "width":4.0,"style":"esriSLSSolid"}},
      {"value":"class_f5", "label":"Freeway More than 1.50" , "symbol":{"type":"esriSLS","color":[115,  0,  0], "width":4.0,"style":"esriSLSSolid"}}]

idDisplay = idFromCode(sScenario)
sTitle = "Congestion: " + ScenarioNameFromCode(sScenario) + ' - ' + SeasonNameFromCode(sVCGroup[0:3]) + ' ' + DOWPkNameFromCode(sVCGroup[3:7])

sdf_Forecasts_wVC = pd.DataFrame.merge(sdf_Forecasts,dfs_VC_MaxDirFG_wVCGroups[idDisplay],on='SEGID')

sdf_Forecasts_wVC = sdf_Forecasts_wVC[['SEGID',sVCGroup,'FT','SHAPE']]

#create map centered on Salt Lake
map_seg = gis.map('Salt Lake')

#define sdf layer
sdf_Forecasts_wVC.spatial.plot(map_widget = map_seg,renderer_type='u-a',unique_values=uv,arcade_expression=arcade_expression,default_symbol="",arcade_title="V/C Ratio Range")

#define map characteristics
map_seg.layout.height='800px'
map_seg.legend=True

#map title
display(Markdown('<h2><center>' + sTitle + '</center></h2>'))

#display map
map_seg

<h2><center>Congestion: 2050 RTP - Annual PM - Weekday</center></h2>

MapView(layout=Layout(height='800px', width='100%'), legend=True)

In [13]:
sdf_Forecasts_wVC

,SEGID,AnnWkPMPrd,FT,SHAPE
0,0068_036.0,0.79,2.0,"{""paths"": [[[421160.5499999998, 4476386.34], [..."
1,WFRC_8000,1.12,2.0,"{""paths"": [[[420067.91000000015, 4479534.80000..."
2,0131_001.7,0.67,2.0,"{""paths"": [[[422222.28000000026, 4480359.77], ..."
3,0068_038.1,1.16,2.0,"{""paths"": [[[420067.91000000015, 4479534.80000..."
4,2046_002.3,0.86,4.0,"{""paths"": [[[422575.00519999955, 4478386.86700..."
...,...,...,...,...
3890,MAG_6247,0.49,4.0,"{""paths"": [[[437218.06879999954, 4461802.4725]..."
3891,MAG_6248,1.95,33.0,"{""paths"": [[[435487.7599999998, 4462445.02], [..."
3892,MAG_6249,0.41,2.0,"{""paths"": [[[442185.8794999998, 4440527.9736],..."
3893,MAG_6250,0.67,2.0,"{""paths"": [[[442531.98900000006, 4440253.285],..."


In [14]:
df_VC = pd.read_csv(os.path.join(dirResults, r'1_VC_' + sScenario + '.csv'))

In [15]:
sScenario

'RTP50'

In [16]:
df_VC_filtered = df_VC[(df_VC['SEGID']=='2034_000.0') & (df_VC['VCGroupCode']==sVCGroup[0:7])]
pd.set_option('display.max_columns', None)
df_VC_filtered

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPERCENT,HVPERCENT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,DOWFACFC,SsnGrp,SsnATGroup,SsnVolCls,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,AvgSsnPrdVol,SiteGroupDOW,DOWGroup,DOWFactor,AvgSsnDOWPrdVol,Prd,HrPctOf,VolPkHrPct,SeasonType,VCGroupCode,PrdHrs,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC,PHF,Pk15PCEFlow,Pk15VC
411548,2034_000.0,3,2446.8,Arterial,1.0,D1,1,5,Collector,525,0.5373,PM,0.2482,8.4,4.5,2050,15000,15000,2000.3679,Arterial,StatewideArterial,3-Suburban,0-AllVolumes,0,1000000,SNT,S00-Ann,1.0,2000,WD2,W1-Weekday,1.12,2235,PM,Prd,0.3721,Season,AnnWkPM,3,831.6435,1.0393,774,864,1.47,1.65,0.98,881,1.68
411549,2034_000.0,3,2446.8,Arterial,1.0,D2,1,5,Collector,525,0.4627,PM,0.2482,11.0,5.6,2050,15000,15000,1722.6321,Arterial,StatewideArterial,3-Suburban,0-AllVolumes,0,1000000,SNT,S00-Ann,1.0,1722,WD2,W1-Weekday,1.12,1925,PM,Prd,0.3721,Season,AnnWkPM,3,716.2925,1.0500,673,752,1.28,1.43,0.98,767,1.46


In [17]:
try:
    df_ForecastAdj = pd.read_csv(os.path.join(dirResults, r'ForecastAdj_' + sScenario + '.csv'))
except Exception:
    display ('No forecast adjustment')
    pass

'No forecast adjustment'

In [18]:
try:
    df_ForecastAdj_filtered = df_ForecastAdj[(df_ForecastAdj['SEGID']=='0085_011.3')]
    df_ForecastAdj_filtered
except Exception:
    display ('No forecast adjustment')
    pass  

'No forecast adjustment'

# Create Layers

In [19]:
for (idx, row) in df_Scenarios.iterrows():

    sScenario = row.loc['ScenarioCode']

    idLoc = idFromCode(sScenario)
    display("Congestion: " + ScenarioNameFromCode(sScenario) + ' - ' + SeasonNameFromCode(sVCGroup[0:3]) + ' ' + DOWPkNameFromCode(sVCGroup[3:7]))

    sdf_Forecasts_wVC = pd.DataFrame.merge(sdf_Forecasts,dfs_VC_MaxDirFG_wVCGroups[idLoc],on='SEGID')

    sdf_Forecasts_wVC['Scenario'] = sScenario
    
    outFields = ['Scenario','SEGID','FT','SHAPE'] + list(df_VCGroups_Expanded.VCGroupCode.unique())
    
    sdf_Forecasts_wVC = sdf_Forecasts_wVC[outFields]
    
    sdf_Forecasts_wVC = sdf_Forecasts_wVC.fillna(0)
    #display(sdf_Forecasts_wVC)

    try:
        sdf_Forecasts_wVC.spatial.to_featureclass(location=os.path.join(dirLayers,'VC_' + sScenario + '.shp'))
    except Exception:
        display ('FAIL')
        pass
    
    



'Congestion: 2015 Base - Annual PM - Weekday'

'Congestion: 2019 Base - Annual PM - Weekday'

'Congestion: 2024 TIP - Annual PM - Weekday'

'Congestion: 2030 RTP - Annual PM - Weekday'

'Congestion: 2040 RTP - Annual PM - Weekday'

...

'Congestion: 2050 No Build - Annual PM - Weekday'

'Congestion: 2030 Needs - Annual PM - Weekday'

'Congestion: 2040 Needs - Annual PM - Weekday'

'Congestion: 2050 Needs - Annual PM - Weekday'

'Congestion: 2050 S5 Draft - Annual PM - Weekday'

# Segment Data for WebApp

In [42]:
dfs_VC = []

for (idx, row) in df_Scenarios.iterrows():

    display(row.loc['ScenarioCode'])
    
    df_VC = pd.read_csv(os.path.join(dirResults, r'1_VC_' + row.loc['ScenarioCode'] + '.csv'))
    
    #some older files may still use old column names, rename them here to be consistent moving forward
    df_VC = df_VC.rename(columns={"MDPERCENT": "MDPCT", "HVPERCENT": "HVPCT"})
    
    dfs_VC.append(df_VC)
    

'Base15'

'Base19'

'TIP24'

'RTP30'

'RTP40'

...

'NB50'

'UF30'

'UF40'

'UF50'

'S550'

In [43]:
dfs_VC[2]

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,DOWFACFC,SsnGrp,SsnATGroup,SsnVolCls,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,AvgSsnPrdVol,SiteGroupDOW,DOWGroup,DOWFactor,AvgSsnDOWPrdVol,Prd,HrPctOf,VolPkHrPct,SeasonType,VCGroupCode,PrdHrs,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC,PHF,Pk15PCEFlow,Pk15VC
0,0006_146.9,1,1465.9,Arterial,1.0,D1,1,2,Principal Arterial,1065,0.4747,AM,0.1810,6.7,4.9,2024,0,0,0.0,Arterial,StatewideArterial,1-Rural,UTRuralLow,0,2500,SNP,M01-Jan,0.82,0,WD0,W1-Weekday,0.94,0,AM,Prd,0.4131,Month,JanWkAM,3,0.0000,1.0379,0,0,0.00,0.00,0.88,0,0.00
1,0006_146.9,1,1465.9,Arterial,1.0,D2,1,2,Principal Arterial,1065,0.5253,AM,0.1810,6.1,4.4,2024,0,0,0.0,Arterial,StatewideArterial,1-Rural,UTRuralLow,0,2500,SNP,M01-Jan,0.82,0,WD0,W1-Weekday,0.94,0,AM,Prd,0.4131,Month,JanWkAM,3,0.0000,1.0342,0,0,0.00,0.00,0.88,0,0.00
2,0006_149.9,1,1632.9,Arterial,1.0,D1,1,2,Principal Arterial,1065,0.5415,AM,0.1623,17.9,14.4,2024,0,0,0.0,Arterial,StatewideArterial,1-Rural,UTRuralLow,0,2500,SNP,M01-Jan,0.82,0,WD0,W1-Weekday,0.94,0,AM,Prd,0.4131,Month,JanWkAM,3,0.0000,1.1078,0,0,0.00,0.00,0.88,0,0.00
3,0006_149.9,1,1632.9,Arterial,1.0,D2,1,2,Principal Arterial,1065,0.4585,AM,0.1623,20.4,16.8,2024,0,0,0.0,Arterial,StatewideArterial,1-Rural,UTRuralLow,0,2500,SNP,M01-Jan,0.82,0,WD0,W1-Weekday,0.94,0,AM,Prd,0.4131,Month,JanWkAM,3,0.0000,1.1248,0,0,0.00,0.00,0.88,0,0.00
4,0006_150.6,1,1848.0,Arterial,1.0,D1,1,2,Principal Arterial,1065,0.5675,AM,0.1617,16.4,13.0,2024,0,0,0.0,Arterial,StatewideArterial,1-Rural,UTRuralLow,0,2500,SNP,M01-Jan,0.82,0,WD0,W1-Weekday,0.94,0,AM,Prd,0.4131,Month,JanWkAM,3,0.0000,1.0978,0,0,0.00,0.00,0.88,0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
770707,1084_001.0,1,884.1,Arterial,1.0,D2,1,5,Collector,572,0.6000,DY,1.0000,24.2,18.3,2024,400,400,240.0,Arterial,RuralRecLowVol,0-AllAreaTypes,0-AllVolumes,0,1000000,SNL,S00-Ann,1.00,240,WD0,7-Sunday,0.94,224,DY,DY,0.1003,Season,AnnSuDY,24,22.4672,1.1399,10,25,0.02,0.04,0.88,29,0.05
770708,2865_001.3,2,2523.7,Arterial,1.0,D1,1,4,Collector,785,0.6000,DY,1.0000,1.7,1.4,2024,0,0,0.0,Arterial,RuralRecLowVol,0-AllAreaTypes,0-AllVolumes,0,1000000,SNL,S00-Ann,1.00,0,WD1,7-Sunday,0.77,0,DY,DY,0.1003,Season,AnnSuDY,24,0.0000,1.0104,0,0,0.00,0.00,0.88,0,0.00
770709,2865_001.3,2,2523.7,Arterial,1.0,D2,1,4,Collector,785,0.6000,DY,1.0000,1.7,1.4,2024,0,0,0.0,Arterial,RuralRecLowVol,0-AllAreaTypes,0-AllVolumes,0,1000000,SNL,S00-Ann,1.00,0,WD1,7-Sunday,0.77,0,DY,DY,0.1003,Season,AnnSuDY,24,0.0000,1.0104,0,0,0.00,0.00,0.88,0,0.00
770710,3004_000.9,2,552.4,Arterial,1.0,D1,1,4,Collector,785,0.6000,DY,1.0000,9.6,8.4,2024,0,0,0.0,Arterial,RuralRecLowVol,0-AllAreaTypes,0-AllVolumes,0,1000000,SNL,S00-Ann,1.00,0,WD1,7-Sunday,0.77,0,DY,DY,0.1003,Season,AnnSuDY,24,0.0000,1.0612,0,0,0.00,0.00,0.88,0,0.00


In [49]:
dReportingColumns_Seg           = ['SEGID'     ,'ForecastAADT'     ,'ForecastAADTAdj'       ]
dReportingColumns_Seg_Names     = ['Segment ID','RTP Forecast AADT','Scenario Forecast AADT']
dReportingColumns_Seg_jsonNames = ['SegID','Forecast','ForecastAdj']

dReportingColumns_Data           = ['VCGroupCode'  ,'FUNCGROUP'       ,'FGFAC'                  ,'SiteGroupSeason'  ,'SeasonFactor' ,'DOWFactor'         ,'PRDFAC'       ,'VolPkHrPct',     'Dir'      ,'DFAC'            ,'MDPCT'         ,'HVPCT'        ,'TrkFac'      ,'PHF','LANES','AREATYPE' ,'FT'             ,'Capacity'      ,'PrdPCEFlow'       ,'PkHrPCEFlow'         ,'Pk15PCEFlow'              ,'PrdVC'     ,'PkHrVC'       ,'Pk15VC'         ]
dReportingColumns_Data_Names     = ['VC Group Code','Functional Group','Functional Group Factor','Season Group Code','Season Factor','Day-of-Week Factor','Period Factor','Max Hour Factor','Direction','Direction Factor','Medium Truck %','Heavy Truck %','Truck Factor','PHF','Lanes','Area Type','Functional Type','Capacity (vph)','Period Flow (vph)','Peak Hour Flow (vph)','Peak 15-Minute Flow (vph)','Period V/C','Peak Hour V/C','Peak 15-Min V/C']
dReportingColumns_Data_jsonNames = ['VCG'          ,'FG'              ,'FGF'                    ,'SG'               ,'SF'           ,'DOW'               ,'PF'           ,'HF'             ,'D'        ,'DF'              ,'MD'            ,'HV'           ,'TF'          ,'PHF','LN'   ,'AT'       ,'FT'             ,'CP'            ,'PrdF'             ,'PkF'                 ,'15F'                      ,'PrdVC'     ,'PkVC'         ,'15VC'           ]

dReportingColumns_All = dReportingColumns_Seg + dReportingColumns_Data
dReportingColumns_All_jsonNames = dReportingColumns_Seg_jsonNames + dReportingColumns_Data_jsonNames
dReportingColumns_All_jsonNames


['SegID',
 'Forecast',
 'ForecastAdj',
 'VCG',
 'FG',
 'FGF',
 'SG',
 'SF',
 'DOW',
 'PF',
 'HF',
 'D',
 'DF',
 'MD',
 'HV',
 'TF',
 'PHF',
 'LN',
 'AT',
 'FT',
 'CP',
 'PrdF',
 'PkF',
 '15F',
 'PrdVC',
 'PkVC',
 '15VC']

In [46]:
exportscenariocode='S550'

for index, df_VC in enumerate(dfs_VC):

    sScenarioCode = df_Scenarios['ScenarioCode'][index]
    
    if (sScenarioCode==exportscenariocode):

        display (sScenarioCode)

        #dfOutputSegs = df_VC.groupby(dReportingColumns_Seg, as_index=False).size().reset_index(name='Freq')
        #dfOutputSegs = dfOutputSegs[dReportingColumns_Seg]

        df_VC['ForecastAADT']    = df_VC.apply(lambda x: "{:,.0f}".format(x['ForecastAADT']), axis=1)
        df_VC['ForecastAADTAdj'] = df_VC.apply(lambda x: "{:,.0f}".format(x['ForecastAADTAdj']), axis=1)

        #dfOutputSegs.columns  = dReportingColumns_Seg_jsonNames
        #dfOutputSegs.to_json(os.path.join(dirResults, 'WebAppData\\segment_vcdata\\_scenariosegs_' + sScenarioCode + ".json"),orient='records')

        df_VC['Capacity']        = df_VC['CAP1HL'] * df_VC['LANES']

        df_VC['FGFAC'          ] = df_VC.apply(lambda x: "{:.3f}" .format(x['FGFAC'          ]), axis=1)    
        df_VC['PRDFAC'         ] = df_VC.apply(lambda x: "{:.3f}" .format(x['PRDFAC'         ]), axis=1)    
        df_VC['VolPkHrPct'     ] = df_VC.apply(lambda x: "{:.3f}" .format(x['VolPkHrPct'     ]), axis=1)    
        df_VC['SeasonFactor'   ] = df_VC.apply(lambda x: "{:.3f}" .format(x['SeasonFactor'   ]), axis=1)
        df_VC['DOWFactor'      ] = df_VC.apply(lambda x: "{:.3f}" .format(x['DOWFactor'      ]), axis=1)
        df_VC['DFAC'           ] = df_VC.apply(lambda x: "{:.3f}" .format(x['DFAC'           ]), axis=1)

        df_VC['Capacity'       ] = round((df_VC['Capacity'   ]/10),0)*10
        df_VC['PrdPCEFlow'     ] = round((df_VC['PrdPCEFlow' ]/10),0)*10
        df_VC['PkHrPCEFlow'    ] = round((df_VC['PkHrPCEFlow']/10),0)*10
        df_VC['Pk15PCEFlow'    ] = round((df_VC['Pk15PCEFlow']/10),0)*10

        df_VC['Capacity'       ] = df_VC.apply(lambda x: "{:,.0f}".format(x['Capacity'   ]), axis=1)
        df_VC['PrdPCEFlow'     ] = df_VC.apply(lambda x: "{:,.0f}".format(x['PrdPCEFlow' ]), axis=1)
        df_VC['PkHrPCEFlow'    ] = df_VC.apply(lambda x: "{:,.0f}".format(x['PkHrPCEFlow']), axis=1)
        df_VC['Pk15PCEFlow'    ] = df_VC.apply(lambda x: "{:,.0f}".format(x['Pk15PCEFlow']), axis=1)

        df_VC['MDPCT'          ] = df_VC.apply(lambda x: "{:.1f}".format(x['MDPCT']), axis=1)
        df_VC['HVPCT'          ] = df_VC.apply(lambda x: "{:.1f}".format(x['HVPCT']), axis=1)

        df_VC['TrkFac'         ] = df_VC.apply(lambda x: "{:.3f}".format(x['TrkFac']), axis=1)
        df_VC['PrdVC'          ] = df_VC.apply(lambda x: "{:.2f}".format(x['PrdVC' ]), axis=1)
        df_VC['PkHrVC'         ] = df_VC.apply(lambda x: "{:.2f}".format(x['PkHrVC']), axis=1)
        df_VC['Pk15VC'         ] = df_VC.apply(lambda x: "{:.2f}".format(x['Pk15VC']), axis=1)  

    #    #fVCforSet = df_VC.groupby(['SEGID','VCGroupCode'], as_index=False).size().reset_index(name='Freq')
    #    
    #    for index, row in dfOutputSegs.iterrows():
    #
    #        sSegID = row['SegID']
    #        #sVCGroupCode = row['VCGroupCode']
    #          
    #        #dfOutputData = df_VC[(df_VC['SEGID'] == sSegID) & (df_VC['VCGroupCode'] == sVCGroupCode)]
    #        
    #        dfOutputData = df_VC[(df_VC['SEGID'] == sSegID)]
    #
    #        #sFilename = sScenarioCode + '_' + sSegID + '_ '+ sVCGroupCode + ".json"
    #        sFilename = sScenarioCode + '_' + sSegID + ".json"
    #        #display(sFilename)
    #        
    #        dfOutputData = dfOutputData[dReportingColumns_Data]
    #        
    #        dfOutputData.columns  = dReportingColumns_Data_jsonNames
    #        
    #        dfOutputData.to_json(os.path.join(dirResults, "WebAppData\\segment_vcdata\\" + sFilename),orient='records')
    

'S550'

In [47]:
df_VC

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,DOWFACFC,SsnGrp,SsnATGroup,SsnVolCls,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,AvgSsnPrdVol,SiteGroupDOW,DOWGroup,DOWFactor,AvgSsnDOWPrdVol,Prd,HrPctOf,VolPkHrPct,SeasonType,VCGroupCode,PrdHrs,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC,PHF,Pk15PCEFlow,Pk15VC,Capacity
0,0006_146.9,1,1814.0,Arterial,1.000,D1,1,2,Principal Arterial,1065,0.574,AM,0.188,5.4,3.9,2050,"2,100","2,100",227.018232,Arterial,StatewideArterial,1-Rural,UTRuralLow,0,2500,SNP,M01-Jan,0.820,185,WD0,W1-Weekday,0.940,173,AM,Prd,0.413,Month,JanWkAM,3,71.4663,1.030,60,70,0.06,0.07,0.88,80,0.08,"1,060"
1,0006_146.9,1,1814.0,Arterial,1.000,D2,1,2,Principal Arterial,1065,0.426,AM,0.188,7.2,5.3,2050,"2,100","2,100",168.621768,Arterial,StatewideArterial,1-Rural,UTRuralLow,0,2500,SNP,M01-Jan,0.820,137,WD0,W1-Weekday,0.940,128,AM,Prd,0.413,Month,JanWkAM,3,52.8768,1.041,40,60,0.04,0.05,0.88,60,0.06,"1,060"
2,0147_003.1,1,861.8,Arterial,1.000,D1,1,3,Minor Arterial,983,0.505,AM,0.140,24.3,35.6,2050,"1,200","1,200",84.967848,Arterial,StatewideArterial,1-Rural,UTRuralLow,0,2500,SNP,M01-Jan,0.820,69,WD0,W1-Weekday,0.940,64,AM,Prd,0.413,Month,JanWkAM,3,26.4384,1.227,30,30,0.03,0.03,0.88,40,0.04,980
3,0147_003.1,1,861.8,Arterial,1.000,D2,1,3,Minor Arterial,983,0.495,AM,0.140,22.6,33.8,2050,"1,200","1,200",83.152152,Arterial,StatewideArterial,1-Rural,UTRuralLow,0,2500,SNP,M01-Jan,0.820,67,WD0,W1-Weekday,0.940,62,AM,Prd,0.413,Month,JanWkAM,3,25.6122,1.214,20,30,0.03,0.03,0.88,40,0.04,980
4,0147_003.9,1,818.1,Arterial,1.000,D1,1,3,Minor Arterial,983,0.511,AM,0.141,24.2,35.5,2050,"1,500","1,500",108.340680,Arterial,StatewideArterial,1-Rural,UTRuralLow,0,2500,SNP,M01-Jan,0.820,88,WD0,W1-Weekday,0.940,82,AM,Prd,0.413,Month,JanWkAM,3,33.8742,1.226,30,40,0.03,0.04,0.88,50,0.05,980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
837211,1084_001.0,1,634.3,Arterial,1.000,D2,1,5,Collector,572,0.600,DY,1.000,21.8,19.2,2050,800,800,480.000000,Arterial,RuralRecLowVol,0-AllAreaTypes,0-AllVolumes,0,1000000,SNL,S00-Ann,1.000,480,WD0,7-Sunday,0.940,449,DY,DY,0.100,Season,AnnSuDY,24,45.0347,1.140,20,50,0.04,0.09,0.88,60,0.10,570
837212,2865_001.3,2,3596.7,Arterial,1.000,D1,1,4,Collector,785,0.600,DY,1.000,1.7,1.6,2050,"3,800","3,800",2280.000000,Arterial,RuralRecLowVol,0-AllAreaTypes,0-AllVolumes,0,1000000,SNL,S00-Ann,1.000,2280,WD1,7-Sunday,0.770,1762,DY,DY,0.100,Season,AnnSuDY,24,176.7286,1.011,70,180,0.09,0.23,0.88,200,0.26,780
837213,2865_001.3,2,3596.7,Arterial,1.000,D2,1,4,Collector,785,0.600,DY,1.000,1.7,1.6,2050,"3,800","3,800",2280.000000,Arterial,RuralRecLowVol,0-AllAreaTypes,0-AllVolumes,0,1000000,SNL,S00-Ann,1.000,2280,WD1,7-Sunday,0.770,1762,DY,DY,0.100,Season,AnnSuDY,24,176.7286,1.011,70,180,0.09,0.23,0.88,200,0.26,780
837214,3004_000.9,3,15605.8,Arterial,1.000,D1,1,3,Minor Arterial,894,0.600,DY,1.000,19.1,7.0,2050,"35,500","35,000",21000.000000,Arterial,RuralRecLowVol,0-AllAreaTypes,0-AllVolumes,0,1000000,SNL,S00-Ann,1.000,21000,WD2,7-Sunday,0.570,11886,DY,DY,0.100,Season,AnnSuDY,24,1192.1658,1.073,530,"1,280",0.59,1.43,0.98,"1,300",1.46,890


In [50]:
for index, df_VC in enumerate(dfs_VC):

    sScenarioCode = df_Scenarios['ScenarioCode'][index]
    
    if (sScenarioCode==exportscenariocode):

        display (sScenarioCode)
        
        dfOutputSegs = df_VC.groupby(dReportingColumns_Seg, as_index=False).size().reset_index(name='Freq')

        for index, row in dfOutputSegs.iterrows():

            sSegID = row['SEGID']

            dfOutputData = df_VC[(df_VC['SEGID'] == sSegID)]

            dfOutputDataNew = dfOutputData[dReportingColumns_All].copy()
            dfOutputDataNew.columns = dReportingColumns_All_jsonNames

            sFilename = sScenarioCode + '_' + sSegID + ".json"

    #        group = merge_hr_anr.groupby('EMPCODE').groups
    #        d = {'EMPCODE': list(group.keys())[0], 'Indicators': [{'IndicatorName': merge_hr_anr.loc[i, 'Indicator']} for i in list(group.values())[0].unique()]}
    #        d['Indicators'] = list(map(dict,sorted(set(map(lambda x: tuple(x.items()),d['Indicators'])), key=list(map(lambda x: tuple(x.items()),d['Indicators'])).index)))
    #        d['Performance'] = [{i['IndicatorName']: merge_hr_anr.loc[merge_hr_anr['Indicator'].eq(i['IndicatorName']), 'CustNAME'].dropna().tolist()} for i in d['Indicators']]
    #        print(d)
    #        
            j = (dfOutputDataNew.groupby(dReportingColumns_Seg_jsonNames, as_index=False)
                    .apply(lambda x: x[dReportingColumns_Data_jsonNames]
                    .to_dict('r'))
                    .reset_index()
                    .rename(columns={0:'VCData'})
                    .to_json(os.path.join(dirResults, "WebAppData\\segment_vcdata\\" + sFilename),orient='records'))#, indent=2))

            #
            #display(j)

            #break


'S550'

In [51]:
dfOutputDataNew

,SegID,Forecast,ForecastAdj,VCG,FG,FGF,SG,SF,DOW,PF,HF,D,DF,MD,HV,TF,PHF,LN,AT,FT,CP,PrdF,PkF,15F,PrdVC,PkVC,15VC
3594,WFRC_8219,"13,500","13,000",JanWkAM,Arterial,1.000,SNS,0.850,1.010,0.180,0.405,D1,0.513,15.1,6.4,1.062,0.88,1,2,3,940,370,440,500,0.39,0.47,0.54
3595,WFRC_8219,"13,500","13,000",JanWkAM,Arterial,1.000,SNS,0.850,1.010,0.180,0.405,D2,0.487,15.1,6.5,1.063,0.88,1,2,3,940,350,420,480,0.37,0.45,0.51
11802,WFRC_8219,"13,500","13,000",FebWkAM,Arterial,1.000,SNS,0.900,1.010,0.180,0.409,D1,0.513,15.1,6.4,1.062,0.88,1,2,3,940,380,470,540,0.41,0.50,0.57
11803,WFRC_8219,"13,500","13,000",FebWkAM,Arterial,1.000,SNS,0.900,1.010,0.180,0.409,D2,0.487,15.1,6.5,1.063,0.88,1,2,3,940,370,450,510,0.39,0.48,0.54
20010,WFRC_8219,"13,500","13,000",MarWkAM,Arterial,1.000,SNS,0.980,1.010,0.180,0.404,D1,0.513,15.1,6.4,1.062,0.88,1,2,3,940,420,510,580,0.45,0.55,0.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
816187,WFRC_8219,"13,500","13,000",AnnFrPM,Arterial,1.000,SNS,1.000,1.140,1.000,0.091,D2,0.600,17.3,7.4,1.072,0.91,1,2,3,940,"3,180",870,960,3.39,0.93,1.02
824394,WFRC_8219,"13,500","13,000",AnnSaDY,Arterial,1.000,SNS,1.000,1.040,1.000,0.080,D1,0.600,17.3,7.4,1.072,0.88,1,2,3,940,360,690,780,0.38,0.73,0.84
824395,WFRC_8219,"13,500","13,000",AnnSaDY,Arterial,1.000,SNS,1.000,1.040,1.000,0.080,D2,0.600,17.3,7.4,1.072,0.88,1,2,3,940,360,690,780,0.38,0.73,0.84
832602,WFRC_8219,"13,500","13,000",AnnSuDY,Arterial,1.000,SNS,1.000,0.770,1.000,0.093,D1,0.600,17.3,7.4,1.072,0.88,1,2,3,940,270,600,680,0.29,0.64,0.73


In [52]:
dfOutputSegs

,SEGID,ForecastAADT,ForecastAADTAdj,Freq
0,0006_146.9,"2,100","2,100",204
1,0006_149.9,"4,400","4,300",204
2,0006_150.6,"4,400","4,300",204
3,0006_152.6,"4,500","4,300",204
4,0006_152.9,"6,600","6,400",204
...,...,...,...,...
3883,WFRC_8215,"11,500","11,500",204
3884,WFRC_8216,"23,500","24,500",204
3885,WFRC_8217,"19,500","19,000",204
3886,WFRC_8218,"37,500","37,000",204


# Seg WebApp Data

In [54]:
df_Scenarios

,Process,ScenarioCode,ScenarioGroup,ScenarioName,ScenarioYear,SegSummaryCSV,PrvScenarioCode,AdjScenarioCode
0,Yes,Base15,Base,2015 Base,2015,v831_SE15_Net15_Summary_SEGID_Detailed.csv,NaN,NaN
1,No,Base19,Base,2019 Base,2019,v831_SE19_Net19_Summary_SEGID_Detailed.csv,Base15,NaN
2,No,TIP24,TIP,2024 TIP,2024,v831_SE24_Net24_Summary_SEGID_Detailed.csv,Base19,NaN
3,No,RTP30,RTP,2030 RTP,2030,v831_SE30_Net30_Summary_SEGID_Detailed.csv,TIP24,NaN
4,No,RTP40,RTP,2040 RTP,2040,v831_SE40_Net40_Summary_SEGID_Detailed.csv,RTP30,NaN
5,No,RTP50,RTP,2050 RTP,2050,v831_SE50_Net50_Summary_SEGID_Detailed.csv,RTP40,NaN
6,No,NB30,No Build,2030 No Build,2030,v831_SE30_Net24_Summary_SEGID_Detailed.csv,TIP24,RTP30
7,No,NB40,No Build,2040 No Build,2040,v831_SE40_Net24_Summary_SEGID_Detailed.csv,NB30,RTP40
8,No,NB50,No Build,2050 No Build,2050,v831_SE50_Net24_Summary_SEGID_Detailed.csv,NB40,RTP50
9,No,UF30,Needs,2030 Needs,2030,v831_SE30_Net30UF_Summary_SEGID_Detailed.csv,TIP24,RTP30


In [60]:
#column names from TDM
nameSegID   = 'SEGID'
nameFG      = 'FUNCGROUP'
nameLinks   = 'LINKS'
nameLanes   = 'LANES'
nameFT      = 'FT'

#column for scenario year
nameSID = 'ScenarioID'
nameS =   'ScenarioName'
nameSYr = 'ScenarioYear'

nameDir     = 'Dir'

indexMelt   = [nameSegID,nameFG,'D1_' + nameFT,'D2_' + nameFT,'D1_' + nameLanes,'D2_' + nameLanes]

df_TDMcmb = pd.DataFrame() 

for (idx, row) in df_Scenarios.iterrows():

    print(row.loc[nameS] + ' CSV Importing')

    csvFileName = os.path.join(dirProcessed, row.SegSummaryCSV)
    #csvFileName = os.path.splitext(dbfFileName)[0]+'.csv'

    df_TDM_import = pd.read_csv(csvFileName)

    #Remove where Total FG
    df_TDM_import = df_TDM_import[df_TDM_import.FUNCGROUP != 'Total']
    #display(df_TDM_import)
        
    print(row.loc[nameS] + ' Processing.')

    #normalize data with period/volume type fields
    df_TDM_melt = pd.melt(df_TDM_import, id_vars=indexMelt, value_vars=['D1_LINKS','D2_LINKS'])
    df_TDM_melt[nameDir] = df_TDM_melt['variable'].str.get(0) + df_TDM_melt['variable'].str.get(1)
    
    df_TDM_melt = df_TDM_melt.rename(columns={"value": nameLinks})
    
    #display(df_TDM_melt)
    
    #calculate direction level links, lanes, capacity
   
    df_TDM_melt[nameLanes] = np.where(df_TDM_melt[nameDir].eq('D1'), df_TDM_melt['D1_'+nameLanes], df_TDM_melt['D2_'+nameLanes])
    df_TDM_melt[nameFT] = np.where(df_TDM_melt[nameDir].eq('D1'), df_TDM_melt['D1_'+nameFT], df_TDM_melt['D2_'+nameFT])
    
    df_TDM_melt = df_TDM_melt.drop(columns=['variable','D1_'+nameLanes,'D2_'+nameLanes,'D1_'+nameFT,'D2_'+nameFT])
    #display(df_TDM_melt)

    df_TDM_melt['ScenarioCode'] = row.loc['ScenarioCode']
    
    df_TDM_meltn = df_TDM_melt[['ScenarioCode','SEGID','FUNCGROUP','Dir','FT','LANES']]
    
    df_TDMcmb = pd.concat([df_TDMcmb,df_TDM_meltn],ignore_index=True)
    
print('Done')

2015 Base CSV Importing
2015 Base Processing.
2019 Base CSV Importing
2019 Base Processing.
2024 TIP CSV Importing
2024 TIP Processing.
2030 RTP CSV Importing
2030 RTP Processing.
2040 RTP CSV Importing
2040 RTP Processing.
2050 RTP CSV Importing
2050 RTP Processing.
2030 No Build CSV Importing
2030 No Build Processing.
2040 No Build CSV Importing
2040 No Build Processing.
2050 No Build CSV Importing
2050 No Build Processing.
2030 Needs CSV Importing
2030 Needs Processing.
2040 Needs CSV Importing
2040 Needs Processing.
2050 Needs CSV Importing
2050 Needs Processing.
2050 S5 Draft CSV Importing
2050 S5 Draft Processing.
Done


In [61]:
df_TDMcmb

,ScenarioCode,SEGID,FUNCGROUP,Dir,FT,LANES
0,Base15,0006_146.9,Total,D1,2.0,1.0
1,Base15,0006_146.9,Arterial,D1,2.0,1.0
2,Base15,0006_146.9,Freeway,D1,0.0,0.0
3,Base15,0006_146.9,Managed,D1,0.0,0.0
4,Base15,0006_146.9,CD Road,D1,0.0,0.0
...,...,...,...,...,...,...
481511,S550,WFRC_8218,CD Road,D2,31.0,1.0
481512,S550,WFRC_8219,Arterial,D2,3.0,1.0
481513,S550,WFRC_8219,Freeway,D2,0.0,0.0
481514,S550,WFRC_8219,Managed,D2,0.0,0.0


In [62]:
df_SegIDsAll = df_TDMcmb.groupby(['SEGID'], as_index=False).agg({'ScenarioCode':[np.size]})
df_SegIDsAll.columns = ['SEGID','numRecords']
df_SegIDsAll


,SEGID,numRecords
0,0006_146.9,128
1,0006_149.9,128
2,0006_150.6,128
3,0006_152.6,128
4,0006_152.9,128
...,...,...
3928,WFRC_8215,128
3929,WFRC_8216,108
3930,WFRC_8217,108
3931,WFRC_8218,128


In [91]:
import itertools

#hours within period, for capacity
dFG = {
     'FUNCGROUP'     : ['Arterial','Freeway','Managed','CD Road']
}
df_FG = pd.DataFrame(dFG)

#hours within period, for capacity
dDir = {
     'Dir' : ['D1','D2']
}
df_Dir = pd.DataFrame(dDir)


#create df with all possible season and day-of-weak/peak groups
df_SegIDsFGAll = pd.DataFrame(list(itertools.product(df_SegIDsAll['SEGID'],df_FG['FUNCGROUP'],df_Scenarios['ScenarioCode'],df_Dir['Dir'])),columns=['SEGID','FUNCGROUP','ScenarioCode','Dir'])
display(df_SegIDsFGAll)

df_ScSegFGDir = pd.DataFrame.merge(df_SegIDsFGAll,df_TDMcmb,on=('ScenarioCode','SEGID','FUNCGROUP','Dir'),how='left')
display(df_ScSegFGDir[df_ScSegFGDir['ScenarioCode']=='S550'])

,SEGID,FUNCGROUP,ScenarioCode,Dir
0,0006_146.9,Arterial,Base15,D1
1,0006_146.9,Arterial,Base15,D2
2,0006_146.9,Arterial,Base19,D1
3,0006_146.9,Arterial,Base19,D2
4,0006_146.9,Arterial,TIP24,D1
...,...,...,...,...
409027,WFRC_8219,CD Road,UF40,D2
409028,WFRC_8219,CD Road,UF50,D1
409029,WFRC_8219,CD Road,UF50,D2
409030,WFRC_8219,CD Road,S550,D1


,SEGID,FUNCGROUP,ScenarioCode,Dir,FT,LANES
24,0006_146.9,Arterial,S550,D1,2.0,1.0
25,0006_146.9,Arterial,S550,D2,2.0,1.0
50,0006_146.9,Freeway,S550,D1,0.0,0.0
51,0006_146.9,Freeway,S550,D2,0.0,0.0
76,0006_146.9,Managed,S550,D1,0.0,0.0
...,...,...,...,...,...,...
408979,WFRC_8219,Freeway,S550,D2,0.0,0.0
409004,WFRC_8219,Managed,S550,D1,0.0,0.0
409005,WFRC_8219,Managed,S550,D2,0.0,0.0
409030,WFRC_8219,CD Road,S550,D1,0.0,0.0


In [64]:
df_ScSegFGDir.groupby(['ScenarioCode'],as_index=False).agg({'SEGID':[np.size]})

,ScenarioCode,SEGID
,,size
0,Base15,31464
1,Base19,31464
2,NB30,31464
3,NB40,31464
4,NB50,31464
5,RTP30,31464
6,RTP40,31464
7,RTP50,31464
8,S550,31464


In [97]:
df_ScSegFGDir_nonbase = df_ScSegFGDir[df_ScSegFGDir['ScenarioCode']!='Base19']
display(df_ScSegFGDir_nonbase)

,SEGID,FUNCGROUP,ScenarioCode,Dir,FT,LANES
0,0006_146.9,Arterial,Base15,D1,NaN,NaN
1,0006_146.9,Arterial,Base15,D2,NaN,NaN
4,0006_146.9,Arterial,TIP24,D1,NaN,NaN
5,0006_146.9,Arterial,TIP24,D2,NaN,NaN
6,0006_146.9,Arterial,RTP30,D1,NaN,NaN
...,...,...,...,...,...,...
409027,WFRC_8219,CD Road,UF40,D2,NaN,NaN
409028,WFRC_8219,CD Road,UF50,D1,NaN,NaN
409029,WFRC_8219,CD Road,UF50,D2,NaN,NaN
409030,WFRC_8219,CD Road,S550,D1,0.0,0.0


In [66]:
df_ScenariosToPrevious = df_Scenarios[['ScenarioCode','PrvScenarioCode']]
df_ScSegFGDir_nonbase_wpreviouscode = pd.DataFrame.merge(df_ScSegFGDir_nonbase,df_ScenariosToPrevious,on='ScenarioCode')
display(df_ScSegFGDir_nonbase_wpreviouscode)

,SEGID,FUNCGROUP,ScenarioCode,Dir,FT,LANES,PrvScenarioCode
0,0006_146.9,Arterial,Base15,D1,NaN,NaN,NaN
1,0006_146.9,Arterial,Base15,D2,NaN,NaN,NaN
2,0006_146.9,Freeway,Base15,D1,NaN,NaN,NaN
3,0006_146.9,Freeway,Base15,D2,NaN,NaN,NaN
4,0006_146.9,Managed,Base15,D1,NaN,NaN,NaN
...,...,...,...,...,...,...,...
377563,WFRC_8219,Freeway,S550,D2,0.0,0.0,NaN
377564,WFRC_8219,Managed,S550,D1,0.0,0.0,NaN
377565,WFRC_8219,Managed,S550,D2,0.0,0.0,NaN
377566,WFRC_8219,CD Road,S550,D1,0.0,0.0,NaN


In [67]:
df_ScSegFGDir_nonbase_wpreviouscode[df_ScSegFGDir_nonbase_wpreviouscode['PrvScenarioCode']==0]

,SEGID,FUNCGROUP,ScenarioCode,Dir,FT,LANES,PrvScenarioCode


In [68]:
df_ScSegFGDir_nonbase_wpreviousdata = pd.DataFrame.merge(df_ScSegFGDir_nonbase_wpreviouscode,df_ScSegFGDir,left_on=('SEGID','FUNCGROUP','PrvScenarioCode','Dir'),right_on=('SEGID','FUNCGROUP','ScenarioCode','Dir'),how='left')
df_ScSegFGDir_nonbase_wpreviousdata = df_ScSegFGDir_nonbase_wpreviousdata[['SEGID','ScenarioCode_x','ScenarioCode_y','FUNCGROUP','Dir','FT_x','FT_y','LANES_x','LANES_y']]
df_ScSegFGDir_nonbase_wpreviousdata = df_ScSegFGDir_nonbase_wpreviousdata.fillna(0)
display(df_ScSegFGDir_nonbase_wpreviousdata)

,SEGID,ScenarioCode_x,ScenarioCode_y,FUNCGROUP,Dir,FT_x,FT_y,LANES_x,LANES_y
0,0006_146.9,Base15,0,Arterial,D1,0.0,0.0,0.0,0.0
1,0006_146.9,Base15,0,Arterial,D2,0.0,0.0,0.0,0.0
2,0006_146.9,Base15,0,Freeway,D1,0.0,0.0,0.0,0.0
3,0006_146.9,Base15,0,Freeway,D2,0.0,0.0,0.0,0.0
4,0006_146.9,Base15,0,Managed,D1,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
377563,WFRC_8219,S550,0,Freeway,D2,0.0,0.0,0.0,0.0
377564,WFRC_8219,S550,0,Managed,D1,0.0,0.0,0.0,0.0
377565,WFRC_8219,S550,0,Managed,D2,0.0,0.0,0.0,0.0
377566,WFRC_8219,S550,0,CD Road,D1,0.0,0.0,0.0,0.0


In [69]:
df_ScSegFGDir_nonbase_wpreviousdata[(df_ScSegFGDir_nonbase_wpreviousdata['ScenarioCode_y']==0)]

,SEGID,ScenarioCode_x,ScenarioCode_y,FUNCGROUP,Dir,FT_x,FT_y,LANES_x,LANES_y
0,0006_146.9,Base15,0,Arterial,D1,0.0,0.0,0.0,0.0
1,0006_146.9,Base15,0,Arterial,D2,0.0,0.0,0.0,0.0
2,0006_146.9,Base15,0,Freeway,D1,0.0,0.0,0.0,0.0
3,0006_146.9,Base15,0,Freeway,D2,0.0,0.0,0.0,0.0
4,0006_146.9,Base15,0,Managed,D1,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
377563,WFRC_8219,S550,0,Freeway,D2,0.0,0.0,0.0,0.0
377564,WFRC_8219,S550,0,Managed,D1,0.0,0.0,0.0,0.0
377565,WFRC_8219,S550,0,Managed,D2,0.0,0.0,0.0,0.0
377566,WFRC_8219,S550,0,CD Road,D1,0.0,0.0,0.0,0.0


In [70]:
df_ScSegFGDir_nonbase_wpreviousdata['Change_FT'] = df_ScSegFGDir_nonbase_wpreviousdata['FT_x'      ] - df_ScSegFGDir_nonbase_wpreviousdata['FT_y'      ]
df_ScSegFGDir_nonbase_wpreviousdata['Change_LN'] = df_ScSegFGDir_nonbase_wpreviousdata['LANES_x'   ] - df_ScSegFGDir_nonbase_wpreviousdata['LANES_y'   ]
#df_VCcmb_scenario_wpreviousdata[['Change_AT','Change_FT','Change_LN']] = df_VCcmb_scenario_wpreviousdata[['Change_AT','Change_FT','Change_LN']].mask(df_VCcmb_scenario_wpreviousdata[['Change_AT','Change_FT','Change_LN']] < 0, 0)
df_ScSegFGDir_nonbase_wpreviousdata

,SEGID,ScenarioCode_x,ScenarioCode_y,FUNCGROUP,Dir,FT_x,FT_y,LANES_x,LANES_y,Change_FT,Change_LN
0,0006_146.9,Base15,0,Arterial,D1,0.0,0.0,0.0,0.0,0.0,0.0
1,0006_146.9,Base15,0,Arterial,D2,0.0,0.0,0.0,0.0,0.0,0.0
2,0006_146.9,Base15,0,Freeway,D1,0.0,0.0,0.0,0.0,0.0,0.0
3,0006_146.9,Base15,0,Freeway,D2,0.0,0.0,0.0,0.0,0.0,0.0
4,0006_146.9,Base15,0,Managed,D1,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
377563,WFRC_8219,S550,0,Freeway,D2,0.0,0.0,0.0,0.0,0.0,0.0
377564,WFRC_8219,S550,0,Managed,D1,0.0,0.0,0.0,0.0,0.0,0.0
377565,WFRC_8219,S550,0,Managed,D2,0.0,0.0,0.0,0.0,0.0,0.0
377566,WFRC_8219,S550,0,CD Road,D1,0.0,0.0,0.0,0.0,0.0,0.0


In [71]:
#delete lines with all zeros

#df_Changes = df_ScSegFGDir_nonbase_wpreviousdata.copy()

df_Changes = df_ScSegFGDir_nonbase_wpreviousdata[(df_ScSegFGDir_nonbase_wpreviousdata['Change_FT']!=0) | (df_ScSegFGDir_nonbase_wpreviousdata['Change_LN']!=0)].copy()

#df_ScSegFGDir_nonbase_wpreviousdata

In [72]:
df_Changes['Change_LN_Text'] = df_Changes['Change_LN'].abs().astype(str)
df_Changes['Change_LN_Text'] = df_Changes['Change_LN_Text'].replace(to_replace = "\.0+$",value = "", regex = True)
df_Changes['Change_LN_Description'] = df_Changes.apply(lambda row: row['Change_LN_Text'] + " " + row['FUNCGROUP'].lower() + " lanes added" if row['Change_LN'] > 1 else "1 " + row['FUNCGROUP'].lower() + " lane added" if row['Change_LN'] == 1 else row['Change_LN_Text'] + " " + row['FUNCGROUP'].lower() + " lanes removed" if row['Change_LN'] < -1 else "1 " + row['FUNCGROUP'].lower() + " lane removed" if row['Change_LN'] == -1 else "", axis=1)

In [73]:
df_Changes[df_Changes['Change_LN']<0]

,SEGID,ScenarioCode_x,ScenarioCode_y,FUNCGROUP,Dir,FT_x,FT_y,LANES_x,LANES_y,Change_FT,Change_LN,Change_LN_Text,Change_LN_Description


In [74]:
df_Changes[df_Changes['SEGID']=='0089_398.7']

,SEGID,ScenarioCode_x,ScenarioCode_y,FUNCGROUP,Dir,FT_x,FT_y,LANES_x,LANES_y,Change_FT,Change_LN,Change_LN_Text,Change_LN_Description
350376,0089_398.7,S550,0,Arterial,D1,4.0,0.0,1.0,0.0,4.0,1.0,1,1 arterial lane added
350377,0089_398.7,S550,0,Arterial,D2,4.0,0.0,1.0,0.0,4.0,1.0,1,1 arterial lane added
350378,0089_398.7,S550,0,Freeway,D1,33.0,0.0,3.0,0.0,33.0,3.0,3,3 freeway lanes added
350379,0089_398.7,S550,0,Freeway,D2,33.0,0.0,3.0,0.0,33.0,3.0,3,3 freeway lanes added


In [75]:
df_Changes['Change_FT_Text'] = df_Changes['Change_FT'].abs().astype(str)
df_Changes['Change_FT_Description'] = df_Changes.apply(lambda row: row['FUNCGROUP'].lower() + " operational changes" if (row['Change_FT'] > 0) & (row['FT_y'] > 0) else row['FUNCGROUP'].lower() + " operational changes" if (row['Change_FT'] < 0) & (row['FT_y'] > 0) else "", axis=1)

In [76]:
df_Changes[(df_Changes['Change_FT']<0) & (df_Changes['FT_y']>0)]

,SEGID,ScenarioCode_x,ScenarioCode_y,FUNCGROUP,Dir,FT_x,FT_y,LANES_x,LANES_y,Change_FT,Change_LN,Change_LN_Text,Change_LN_Description,Change_FT_Text,Change_FT_Description


In [77]:
df_Changes_wDirections = pd.DataFrame.merge(df_Changes,df_Directions,on='Dir',how='left')

df_Changes_wDirections['Change_Conjunction'] = df_Changes_wDirections.apply(lambda row: ' and ' if (row['Change_FT_Description'] != '') & (row['Change_LN_Description'] != '') else '', axis=1)

df_Changes_wDirections['Change_Text'] = df_Changes_wDirections['DirDesc'] + ": " + df_Changes_wDirections['Change_LN_Description'] + df_Changes_wDirections['Change_Conjunction']  + df_Changes_wDirections['Change_FT_Description'] 
df_Changes_wDirections

,SEGID,ScenarioCode_x,ScenarioCode_y,FUNCGROUP,Dir,FT_x,FT_y,LANES_x,LANES_y,Change_FT,Change_LN,Change_LN_Text,Change_LN_Description,Change_FT_Text,Change_FT_Description,DirDesc,Change_Conjunction,Change_Text
0,0006_146.9,S550,0,Arterial,D1,2.0,0.0,1.0,0.0,2.0,1.0,1,1 arterial lane added,2.0,,NB/EB,,NB/EB: 1 arterial lane added
1,0006_146.9,S550,0,Arterial,D2,2.0,0.0,1.0,0.0,2.0,1.0,1,1 arterial lane added,2.0,,SB/WB,,SB/WB: 1 arterial lane added
2,0006_149.9,S550,0,Arterial,D1,2.0,0.0,1.0,0.0,2.0,1.0,1,1 arterial lane added,2.0,,NB/EB,,NB/EB: 1 arterial lane added
3,0006_149.9,S550,0,Arterial,D2,2.0,0.0,1.0,0.0,2.0,1.0,1,1 arterial lane added,2.0,,SB/WB,,SB/WB: 1 arterial lane added
4,0006_150.6,S550,0,Arterial,D1,2.0,0.0,1.0,0.0,2.0,1.0,1,1 arterial lane added,2.0,,NB/EB,,NB/EB: 1 arterial lane added
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8197,WFRC_8217,S550,0,Arterial,D2,4.0,0.0,1.0,0.0,4.0,1.0,1,1 arterial lane added,4.0,,SB/WB,,SB/WB: 1 arterial lane added
8198,WFRC_8218,S550,0,CD Road,D1,31.0,0.0,1.0,0.0,31.0,1.0,1,1 cd road lane added,31.0,,NB/EB,,NB/EB: 1 cd road lane added
8199,WFRC_8218,S550,0,CD Road,D2,31.0,0.0,1.0,0.0,31.0,1.0,1,1 cd road lane added,31.0,,SB/WB,,SB/WB: 1 cd road lane added
8200,WFRC_8219,S550,0,Arterial,D1,3.0,0.0,1.0,0.0,3.0,1.0,1,1 arterial lane added,3.0,,NB/EB,,NB/EB: 1 arterial lane added


In [78]:
df_Changes_wDirections[(df_Changes_wDirections['Change_FT']<0) & (df_Changes_wDirections['Change_LN']>0)]

,SEGID,ScenarioCode_x,ScenarioCode_y,FUNCGROUP,Dir,FT_x,FT_y,LANES_x,LANES_y,Change_FT,Change_LN,Change_LN_Text,Change_LN_Description,Change_FT_Text,Change_FT_Description,DirDesc,Change_Conjunction,Change_Text


In [79]:
df_Changes_HTML = df_Changes_wDirections.groupby(['SEGID','ScenarioCode_x'])['Change_Text'].apply('<br/>'.join).reset_index()
df_Changes_HTML = pd.DataFrame.merge(df_Changes_HTML,df_Scenarios,left_on='ScenarioCode_x',right_on='ScenarioCode',how='left')
df_Changes_HTML['Change_HTML'] = "<b>" + df_Changes_HTML['ScenarioName'] + ":</b><br/>" + df_Changes_HTML['Change_Text']
df_Changes_HTML

,SEGID,ScenarioCode_x,Change_Text,Process,ScenarioCode,ScenarioGroup,ScenarioName,ScenarioYear,SegSummaryCSV,PrvScenarioCode,AdjScenarioCode,Change_HTML
0,0006_146.9,S550,NB/EB: 1 arterial lane added<br/>SB/WB: 1 arte...,Yes,S550,Scenario,2050 S5 Draft,2050,v831_SE50_Net50_Summary_SEGID_Detailed_S5Draft...,NaN,RTP50,<b>2050 S5 Draft:</b><br/>NB/EB: 1 arterial la...
1,0006_149.9,S550,NB/EB: 1 arterial lane added<br/>SB/WB: 1 arte...,Yes,S550,Scenario,2050 S5 Draft,2050,v831_SE50_Net50_Summary_SEGID_Detailed_S5Draft...,NaN,RTP50,<b>2050 S5 Draft:</b><br/>NB/EB: 1 arterial la...
2,0006_150.6,S550,NB/EB: 1 arterial lane added<br/>SB/WB: 1 arte...,Yes,S550,Scenario,2050 S5 Draft,2050,v831_SE50_Net50_Summary_SEGID_Detailed_S5Draft...,NaN,RTP50,<b>2050 S5 Draft:</b><br/>NB/EB: 1 arterial la...
3,0006_152.6,S550,NB/EB: 1 arterial lane added<br/>SB/WB: 1 arte...,Yes,S550,Scenario,2050 S5 Draft,2050,v831_SE50_Net50_Summary_SEGID_Detailed_S5Draft...,NaN,RTP50,<b>2050 S5 Draft:</b><br/>NB/EB: 1 arterial la...
4,0006_152.9,S550,NB/EB: 1 arterial lane added<br/>SB/WB: 1 arte...,Yes,S550,Scenario,2050 S5 Draft,2050,v831_SE50_Net50_Summary_SEGID_Detailed_S5Draft...,NaN,RTP50,<b>2050 S5 Draft:</b><br/>NB/EB: 1 arterial la...
...,...,...,...,...,...,...,...,...,...,...,...,...
3922,WFRC_8215,S550,NB/EB: 2 arterial lanes added<br/>SB/WB: 2 art...,Yes,S550,Scenario,2050 S5 Draft,2050,v831_SE50_Net50_Summary_SEGID_Detailed_S5Draft...,NaN,RTP50,<b>2050 S5 Draft:</b><br/>NB/EB: 2 arterial la...
3923,WFRC_8216,S550,NB/EB: 2 freeway lanes added<br/>SB/WB: 2 free...,Yes,S550,Scenario,2050 S5 Draft,2050,v831_SE50_Net50_Summary_SEGID_Detailed_S5Draft...,NaN,RTP50,<b>2050 S5 Draft:</b><br/>NB/EB: 2 freeway lan...
3924,WFRC_8217,S550,NB/EB: 1 arterial lane added<br/>SB/WB: 1 arte...,Yes,S550,Scenario,2050 S5 Draft,2050,v831_SE50_Net50_Summary_SEGID_Detailed_S5Draft...,NaN,RTP50,<b>2050 S5 Draft:</b><br/>NB/EB: 1 arterial la...
3925,WFRC_8218,S550,NB/EB: 1 cd road lane added<br/>SB/WB: 1 cd ro...,Yes,S550,Scenario,2050 S5 Draft,2050,v831_SE50_Net50_Summary_SEGID_Detailed_S5Draft...,NaN,RTP50,<b>2050 S5 Draft:</b><br/>NB/EB: 1 cd road lan...


In [86]:
df_ScSegFGDir_baseNoZero_wDirections

,SEGID,FUNCGROUP,ScenarioCode,LANES,FT,FT_Description,LANES_Text,Dir,DirDesc,FT_Text,FG_LANES


In [84]:
df_ScSegFGDir_base = df_ScSegFGDir[df_ScSegFGDir['ScenarioCode']=='Base19']
#display(df_ScSegFGDir_base)

df_ScSegFGDir_baseNoZero = df_ScSegFGDir_base[df_ScSegFGDir_base['LANES']>0]

df_FTs = pd.read_csv(os.path.join(dirTDM, r'FTs.csv'))
df_ScSegFGDir_baseNoZero = pd.DataFrame.merge(df_ScSegFGDir_baseNoZero,df_FTs,on='FT',how='left')

df_ScSegFGDir_baseNoZero['LANES_Text'] = df_ScSegFGDir_baseNoZero['LANES'].abs().astype(str)
df_ScSegFGDir_baseNoZero['LANES_Text'] = df_ScSegFGDir_baseNoZero['LANES_Text'].replace(to_replace = "\.0+$",value = "", regex = True)

df_ScSegFGDir_baseNoZero['LANES_Text'] = df_ScSegFGDir_baseNoZero.apply(lambda row: row['LANES_Text'] + " lanes" if row['LANES'] > 1 else "1 lane" if row['LANES'] == 1 else "", axis=1)

df_ScSegFGDir_baseNoZero_wDirections = pd.DataFrame.merge(df_ScSegFGDir_baseNoZero,df_Directions,on='Dir',how='left')
display(df_ScSegFGDir_baseNoZero_wDirections)

df_ScSegFGDir_baseNoZero_wDirections['FT_Text'] = df_ScSegFGDir_baseNoZero_wDirections['DirDesc'] + " " + df_ScSegFGDir_baseNoZero_wDirections['FUNCGROUP'] + " (" + df_ScSegFGDir_baseNoZero_wDirections['FT_Description'] + ")"
df_ScSegFGDir_baseNoZero_wDirections = df_ScSegFGDir_baseNoZero_wDirections.fillna("")
df_ScSegFGDir_baseNoZero_wDirections['FG_LANES'] = df_ScSegFGDir_baseNoZero_wDirections['FT_Text']  + ": " + df_ScSegFGDir_baseNoZero_wDirections['LANES_Text']
display(df_ScSegFGDir_baseNoZero_wDirections)

df_ScSegFGDir_baseNoZero_collapse = df_ScSegFGDir_baseNoZero_wDirections.groupby(['SEGID','ScenarioCode'])['FG_LANES'].apply('<br/>'.join).reset_index()

df_ScSegFGDir_baseNoZero_collapse_wScenarioName = pd.DataFrame.merge(df_ScSegFGDir_baseNoZero_collapse, df_Scenarios, on='ScenarioCode',how='left')
df_ScSegFGDir_baseNoZero_collapse_wScenarioName['BaseDesc_HTML'] = "<b>" + df_ScSegFGDir_baseNoZero_collapse_wScenarioName['ScenarioName'] + ":</b><br/>" + df_ScSegFGDir_baseNoZero_collapse_wScenarioName['FG_LANES']

df_ScSegFGDir_baseHTML = df_ScSegFGDir_baseNoZero_collapse_wScenarioName[['SEGID','BaseDesc_HTML']]
display(df_ScSegFGDir_baseHTML)


,SEGID,FUNCGROUP,ScenarioCode,LANES,FT,FT_Description,LANES_Text,Dir,DirDesc


,SEGID,FUNCGROUP,ScenarioCode,LANES,FT,FT_Description,LANES_Text,Dir,DirDesc,FT_Text,FG_LANES


KeyError: 'ScenarioCode'

In [89]:
df_ScSegFGDir_base

,SEGID,FUNCGROUP,ScenarioCode,Dir,FT,LANES
2,0006_146.9,Arterial,Base19,D1,NaN,NaN
3,0006_146.9,Arterial,Base19,D2,NaN,NaN
28,0006_146.9,Freeway,Base19,D1,NaN,NaN
29,0006_146.9,Freeway,Base19,D2,NaN,NaN
54,0006_146.9,Managed,Base19,D1,NaN,NaN
...,...,...,...,...,...,...
408957,WFRC_8219,Freeway,Base19,D2,NaN,NaN
408982,WFRC_8219,Managed,Base19,D1,NaN,NaN
408983,WFRC_8219,Managed,Base19,D2,NaN,NaN
409008,WFRC_8219,CD Road,Base19,D1,NaN,NaN


In [ ]:
df_SegmentDetails = pd.DataFrame()
display(df_SegmentDetails)

for (idx, row) in df_SegIDsAll.iterrows():
    
    _segid = row.loc['SEGID']
    print("SEGID: " + _segid)

    #strFinal = "<b>Segment " + _segid + "</b><br/><br/>"
    strFinal = "<br/>"
    
    for (idx, row) in df_Scenarios.iterrows():
        _scenariocode = row.loc['ScenarioCode']
        _scenarioname = row.loc['ScenarioName']
        #print(_scenariocode)
        #print(_scenarioname)
        
        sText = "<b>" + _scenarioname + "</b><br/>"
        
        #list base conditions for base scenario
        if _scenariocode == 'Base19':
            
            #print(_scenarioname)
            
            df_base = df_ScSegFGDir_baseHTML[df_ScSegFGDir_baseHTML['SEGID']==_segid]
            
            if len(df_base.index) > 0:
                strFinal = strFinal + df_base.iloc[0]['BaseDesc_HTML'] + "<br/><br/>"
                #display(df_base.iloc[0]['BaseDesc_HTML'])
            else:
                strFinal = strFinal + "<b>No lanes in " +_scenarioname + "</b><br/><br/>"
            
            #for (idx, row) in df_base.iterrows():
                
                #print(row.loc['BaseDesc_HTML'])
        else:
            
            df_change = df_Changes_HTML[(df_Changes_HTML['SEGID']==_segid) & (df_Changes_HTML['ScenarioCode_x']==_scenariocode)]
            
            if len(df_change.index) > 0:
                #print(_scenarioname)
                strFinal = strFinal + df_change.iloc[0]['Change_HTML'] + "<br/><br/>" 
                #display(df_change.iloc[0]['Change_HTML'])
    
    #display(strFinal)
    df_SegmentDetails = df_SegmentDetails.append({'SEGID'         : _segid,
                                                  'SegDetail_HTML': strFinal
                                                 },ignore_index=True)
    df_SegmentDetails.index.name = 'ID'
    

In [ ]:
dirResults

In [ ]:
df_SegmentDetails.to_json(os.path.join(dirResults, r'WebAppData\segmentdetails.json'),orient='records')

In [ ]:
len(df_SegmentDetails.index)

# Forecasts Prep

In [106]:
df_Scenarios

,Process,ScenarioCode,ScenarioGroup,ScenarioName,ScenarioYear,SegSummaryCSV,PrvScenarioCode,AdjScenarioCode
0,No,Base15,Base,2015 Base,2015,v831_SE15_Net15_Summary_SEGID_Detailed.csv,NaN,NaN
1,No,Base19,Base,2019 Base,2019,v831_SE19_Net19_Summary_SEGID_Detailed.csv,Base15,NaN
2,No,TIP24,TIP,2024 TIP,2024,v831_SE24_Net24_Summary_SEGID_Detailed.csv,Base19,NaN
3,No,RTP30,RTP,2030 RTP,2030,v831_SE30_Net30_Summary_SEGID_Detailed.csv,TIP24,NaN
4,No,RTP40,RTP,2040 RTP,2040,v831_SE40_Net40_Summary_SEGID_Detailed.csv,RTP30,NaN
5,No,RTP50,RTP,2050 RTP,2050,v831_SE50_Net50_Summary_SEGID_Detailed.csv,RTP40,NaN
6,No,NB30,No Build,2030 No Build,2030,v831_SE30_Net24_Summary_SEGID_Detailed.csv,TIP24,RTP30
7,No,NB40,No Build,2040 No Build,2040,v831_SE40_Net24_Summary_SEGID_Detailed.csv,NB30,RTP40
8,No,NB50,No Build,2050 No Build,2050,v831_SE50_Net24_Summary_SEGID_Detailed.csv,NB40,RTP50
9,No,UF30,Needs,2030 Needs,2030,v831_SE30_Net30UF_Summary_SEGID_Detailed.csv,TIP24,RTP30


In [107]:
df_Forecasts = pd.read_csv(os.path.join(dirResults, r'0_Forecasts.csv'))

df_Forecasts_wScenarios = pd.DataFrame.merge(df_Forecasts,df_Scenarios,on='ScenarioCode')

df_Forecasts_wScenarios['ForecastAADT_Rounded'] = (df_Forecasts_wScenarios['ForecastAADT'] / 100).astype(int) *100

df_Forecasts_wScenarios['Forecast_Formatted'] = df_Forecasts_wScenarios['ForecastAADT_Rounded'].map('{:,.0f}'.format)

df_Forecasts_wScenarios['Forecast_HTML'] = "<b>" + df_Forecasts_wScenarios['ScenarioName'] + "</b>: " + df_Forecasts_wScenarios['Forecast_Formatted']

df_Forecasts_wScenarios = df_Forecasts_wScenarios.groupby(['SEGID'])['Forecast_HTML'].apply('<br/>'.join).reset_index()

display(df_Forecasts_wScenarios)

df_Forecasts_wScenarios.to_json(os.path.join(dirResults, r'WebAppData\forecastdetails.json'),orient='records')


,SEGID,Forecast_HTML
0,0006_000.0,<b>2015 Base</b>: 300<br/><b>2019 Base</b>: 0<...
1,0006_000.7,<b>2015 Base</b>: 300<br/><b>2019 Base</b>: 0<...
2,0006_016.0,<b>2015 Base</b>: 300<br/><b>2019 Base</b>: 0<...
3,0006_046.0,<b>2015 Base</b>: 300<br/><b>2019 Base</b>: 0<...
4,0006_060.2,<b>2015 Base</b>: 300<br/><b>2019 Base</b>: 0<...
...,...,...
8503,WFRC_8215,"<b>2015 Base</b>: 0<br/><b>2019 Base</b>: 3,70..."
8504,WFRC_8216,<b>2015 Base</b>: 0<br/><b>2019 Base</b>: 0<br...
8505,WFRC_8217,<b>2015 Base</b>: 0<br/><b>2019 Base</b>: 0<br...
8506,WFRC_8218,"<b>2015 Base</b>: 0<br/><b>2019 Base</b>: 25,5..."


# Seg File Export

In [ ]:
#for (idx, row) in sdf_Forecasts.iterrows():
#
#    segid = row.loc['SEGID']
#    
#    #print(segid)   
#    
#    df_export = df_VCcmb[(df_VCcmb['SEGID']==segid) & (df_VCcmb['VCGroupCode'].str.contains('DY')==False) & (df_VCcmb['VCGroupCode'].str.contains('FrPM')==False)]
#    df_export = df_export.drop(columns=['SEGID'])
#    
#    df_export_s_fg = df_export.groupby(['ScenarioCode','FUNCGROUP','AREATYPE','FT','LANES','Cap1Hr','DOWFACFC','SsnGrp','ForecastAADT'],as_index=False).agg({'VCGroupCode':[np.size]})
#    df_export_s_fg.columns = ['ScenarioCode','FUNCGROUP','AREATYPE','FT','LANES','Cap1Hr','DOWFACFC','SsnGrp','ForecastAADT','numRec']
#    df_export_s_fg_vc = df_export[['ScenarioCode','FUNCGROUP','VCGroupCode','Dir','SeasonFactor','DOWFactor','PercentTrucksMD','PercentTrucksHV','VolPercentSegDY','PHF','VolMaxHourPercent','ForecastPeakHour','ForecastPeak15MinPCE','VCPeak15MinPCE']]
#    
#    filename_s_fg = os.path.join(dirResults,r"WebAppData\segments\\" + segid + '_s_fg.csv')
#    filename_s_fg_vc = os.path.join(dirResults,r"WebAppData\segments\\" + segid + '_s_fg_vc.csv')
#    display('Exporting ' + segid)
#    df_export_s_fg.to_csv(filename_s_fg,index=False)
#    df_export_s_fg_vc.to_csv(filename_s_fg_vc,index=False)

In [ ]:
df_export

In [ ]:
df_VCcmb_base = df_VCcmb[df_VCcmb['ScenarioCode']=='Base19']
display(df_VCcmb_base)

In [ ]:
df_VCcmb_nonbase = df_VCcmb[df_VCcmb['ScenarioCode']!='Base19']
display(df_VCcmb_nonbase)